In [1]:
################################################
# Ayoub Sikouky
# Title: SISR Using a Multi Model System
# Edit Date: 22/04/2022
################################################
import os
import tensorflow as tf

### 0 : Helper Functions

In [2]:
########################### Data ###########################
def download_data(f_name, dst, source_url):
    """
    Download and unzip datastes.
    #### Parameters:
    f_name: zip file name.
    dst: The destination directory where the files will be stored.
    source_url: The URL for the dataset.
    """
    file_url = os.path.join(source_url, f_name)
    dst_abspath = os.path.abspath(dst)
    tf.keras.utils.get_file(f_name, file_url, cache_subdir=dst_abspath, extract=True)
    # Removing the zip file.
    os.remove(os.path.join(dst_abspath, f_name)) 

### 1: Preparing the Data

In [ ]:
# Downloading the datasets used in training and validation.
DIV2K_URL = "http://data.vision.ee.ethz.ch/cvl/DIV2K/"
DIV2K_FILE_NAMES = ["DIV2K_train_HR.zip", "DIV2K_valid_HR.zip", 
                    "DIV2K_train_LR_bicubic_X2.zip", "DIV2K_valid_LR_bicubic_X2.zip"]
dst_dir = "Original_Datasets/DIV2K"


for file in DIV2K_FILE_NAMES:
    download_data(file, dst_dir, DIV2K_URL)